<a href="https://colab.research.google.com/github/alihojatnia/GNNs/blob/main/Simple_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install dgl 
!pip install dgl
!pip install dgllife
!pip install rdkit

In [20]:
from torch.utils.data import DataLoader
import dgl
import dgllife
from dgllife import data
from dgllife.data import ESOL
from dgllife.utils import CanonicalAtomFeaturizer,CanonicalBondFeaturizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl import DGLGraph

In [19]:
import torch
import torch.nn as nn
import dgl
import dgl.function as fn
import dgllife
from dgllife.utils import CanonicalAtomFeaturizer,CanonicalBondFeaturizer
# Define the node and edge featurizers
node_f = CanonicalAtomFeaturizer()
edge_f =CanonicalBondFeaturizer()

# Load the ESOL dataset
g = dgllife.data.ESOL(node_featurizer=node_f, edge_featurizer=edge_f)


class GNN(nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(GNN, self).__init__()
        self.linear1 = nn.Linear(in_feats, hidden_size)
        self.linear2 = nn.Linear(hidden_size, out_feats)

    def forward(self, g, inputs):
        h = torch.relu(self.linear1(inputs))
        h = self.linear2(h)
        return h


def message_func(edges):
    return {'m': edges.src['h']}


def reduce_func(nodes):
    return {'h': torch.mean(nodes.mailbox['m'], dim=1)}


class Model(nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(Model, self).__init__()
        self.gnn = GNN(in_feats, hidden_size, out_feats)

    def forward(self, g, inputs):
        g.ndata['h'] = inputs
        g.update_all(message_func, reduce_func)
        return dgl.mean_nodes(g, "h")


model = Model(in_feats=74, hidden_size=32, out_feats=1)


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    total_loss = 0
    for i, graph in enumerate(g.graphs):
        optimizer.zero_grad()
        output = model(graph, graph.ndata['h'])
        output.requires_grad = True
        loss = criterion(output, g.labels[i].unsqueeze(0))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print("Epoch {}, Average Loss: {:.4f}".format(epoch, total_loss / len(g.graphs)))



Processing dgl graphs from scratch...
Processing molecule 1000/1128


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 74])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.8/dist-packages/dgl/core.py:84: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  dgl_warning(


Epoch 0, Average Loss: 14.1923
Epoch 1, Average Loss: 14.1869
Epoch 2, Average Loss: 14.1914
Epoch 3, Average Loss: 14.1880
Epoch 4, Average Loss: 14.1911
Epoch 5, Average Loss: 14.1885
Epoch 6, Average Loss: 14.1910
Epoch 7, Average Loss: 14.1888
Epoch 8, Average Loss: 14.1910
Epoch 9, Average Loss: 14.1890
